Alexander S. Lundervold & Arvid Lundervold, 15.09.22

# Using deep learning to extract imaging-biomarkers for brain cancer analyses in MRI of glioma

<img width=40% src="https://upload.wikimedia.org/wikipedia/commons/6/64/Glioblastoma_macro.jpg">

In this notebook, we'll use a deep learning model to segment brain tumors from multi-parametric MRI and then extract features from the resulting tumor masks. Such features can potentially be associated with tumor severity and prognosis and contribute to better treatment. 

Extracting features from objects of interest in medical images for diagnostic purposes is often referred to as **radiomics**. The goal of radiomics is to extract information from medical images that can be used as part of a medical imaging-based diagnostic workflow. The information can be extracted from various imaging modalities, e.g., different MRI contrasts, PET imaging, CT imaging, and so on. One can then combine it with other sources of information (e.g., demographics, clinical data, genetics). In such a way, radiomics–and **radiogenomics**–can open the door to sophisticated and powerful analyses. 

## Radiomics workflow

In this notebook, we will estimate the locations and extent of the brain tumors' T2-enhancing and non-enhancing regions. We can then use this to extract the tumor location and the tumor burden. Additionally, we can look at the features of the MRI images inside each of these two tumor parts. 

![](assets/radiomics.png)

## Tumor segmentation provides tumor volumes and locations

Tumor burden and anatomical locations are highly informative when assessing prognosis and planning treatment in brain tumors. Once we can segment the tumors, we automatically obtain tumor volumes. If we have repeated scans of the same tumors, we obtain estimates of tumor progression. By further analyses one can also estimate the anatomical locations of the tumors. 

### Tumor segmentation

<img src="assets/tumor_segmentation.png">

### Tumor progression

<img src="assets/tumor_volumes.png">

### Tumor localization

Using software that can segment the brain into meaningful anatomical regions (for example, FreeSurfer), one can estimate the overlap between the tumor and these anatomical regions. This provides **tumor localization**, which is of course highly relevant for assessment and prognosis. 

<img src="https://upload.wikimedia.org/wikipedia/commons/9/9e/Brainanim.gif">
<center><small> Freesurfer parcellation and segmentation</small></center>

<img width=60% src="assets/tumor_locations.png">

## Data from TCGA

We'll use data from the TCGA collection:

![](assets/tcga.png)

TMP

DATA: https://www.dropbox.com/t/DkMHIn5XSzmxE0sM

Extract to the folder `data/`

# Setup

As always, we need to import some libraries and decide where to fetch and store data.

In [ ]:
%matplotlib inline

In [ ]:
#%pip install hd_glio

In [ ]:
import os, sys, pandas as pd, numpy as np, random, seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

import nibabel as nib

In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
NB_DIR = Path.cwd()
LOCAL_DATA = NB_DIR/'data'
IMG_DATA = LOCAL_DATA/'TCGA'

# Brain tumor imaging data

We will use a small sample data set from the TCGA collection. For simplicity, we use versions of the MRI images that have already been co-registered and converted to NIfTI format: https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=24282668. This saves us a few steps.

We've prepared a small sample data set containing data from 10 subjects.

> **Note:** The TCGA-GBM collection has data from 262 subjects. We use a small sample for illustration.

In [ ]:
sample_df = pd.read_csv(LOCAL_DATA/'TCGA-sample_data.csv')

In [ ]:
sample_df

For convenience, the corresponding images have already been downloaded:

In [ ]:
sorted(list((IMG_DATA/sample_df['img_dir'].values[0]).iterdir()))

Here are all the subject IDs in our sample data set:

In [ ]:
subject_ids = sample_df['img_dir'].values
subject_ids

## Inspect some images

It's very important to inspect your data. After fetching the data and after performing any transformations on the data. In the case of images, that typically means displaying the images:

In [ ]:
def plot_image(subj_id, slice_nb):
    images = [nib.load(i) for i in sorted(list((IMG_DATA/subj_id).glob("*.nii*")))]
    nb_slices = images[0].shape[-1]
    
    flair, t1, t1c, t2 = [np.flip(np.rot90(i.get_fdata()[:,:,slice_nb])) for i in images]

    f,axs = plt.subplots(1,4, figsize=(18,5))
    axs[0].imshow(flair, cmap="gray")
    axs[0].axis('off')
    axs[1].imshow(t1, cmap='gray')
    axs[1].axis('off')
    axs[2].imshow(t1c, cmap="gray")
    axs[2].axis("off")
    axs[3].imshow(t2, cmap="gray")
    axs[3].axis("off")

In [ ]:
plot_image(subject_ids[0], slice_nb=100)

Understanding the data is such a crucial part of any machine learning project that it's worthwhile to spend a lot of time figuring out ways to construct functions to plot and to inspect features of the data. By using IPyWidgets, we can make our plots interactive:

In [ ]:
from ipywidgets import interactive, IntSlider, Select

In [ ]:
select_layer = IntSlider(value=100, min=0, max=154, 
                         description='Slice', continuous_update=False)

select_subj = Select(options=subject_ids, description="Subject")

In [ ]:
interactive(plot_image, subj_id=select_subj, slice_nb=select_layer)

We should also attempt to understand the _distribution_ of the downloaded data in various ways. We can for example ask what  the variation in mean voxel intensities is for each of the image sequences (whole-brain intensities).

In [ ]:
intensities = {}
for subj_id in subject_ids:
    flair, t1, t1c, t2 = [np.mean(nib.load(i).get_fdata()) for i in sorted(list((IMG_DATA/subj_id).glob("*.nii*")))]
    intensities[subj_id] = [flair, t1, t1c, t2]

In [ ]:
df_intensities = pd.DataFrame.from_dict(intensities, orient='index', 
                                       columns = ['flair', 't1', 't1c', 't2'])

In [ ]:
df_intensities

In [ ]:
sns.boxplot(data=df_intensities)
plt.show()

We observe quite some variation in the mean voxel intensities across the data set. This leads to a need for some kind of normalization.

# Brain tumor segmentation using HD-GLIO

In this case, we will use an existing model that has already been trained on a large collection of MRI examinations (3220 examinations from 1450 brain tumor patients, sourced from multiple centers). The pipeline, including the trained model and various preprocessing steps, is freely available via the code-sharing platform **GitHub** under an **open source license** (Apache-2.0): https://github.com/NeuroAI-HD/HD-GLIO. 

![](assets/open-source.png)

HD-GLIO was constructed as a collaboration between the Department of Neuroradiology at the Heidelberg University Hospital, Germany and the Division of Medical Image Computing at the German Cancer Research Center (DKFZ) Heidelberg, Germany.

## The network architecture: U-Net and the nnU-Net

HD-GLIO is based on a version of the widely used U-Net architecture (also used in our previous notebook). Specifically, it is based on a variant of the open source nnU-Net ("no-new-Net"), available here: https://github.com/MIC-DKFZ/nnUNet. 

**U-Net**

![](https://miro.medium.com/max/1400/1*x0kR2rGlTibVbu8InCNBVg.jpeg)

**nnUNet**

The main point of nnUNet is that there is a lot of performance gains to be had by optimizing the deep learning workflow--preprocessing, postprocessing, etc, rather than creating completely new artificial neural network architectures. You can read all about the ideas here: https://www.nature.com/articles/s41592-020-01008-z.

![](https://media.springernature.com/lw685/springer-static/image/art%3A10.1038%2Fs41592-020-01008-z/MediaObjects/41592_2020_1008_Fig2_HTML.png)

## Running HD-GLIO on our data

We create a small helper function to run HD-GLIO

In [ ]:
def run_hdglio(t1, t1c, t2, flair, output, verbose=True):
    
    if verbose: 
        !CUDA_VISIBLE_DEVICES=0 $sys.exec_prefix/bin/hd_glio_predict -t1 $t1 -t1c $t1c -t2 $t2 -flair $flair -o $output
    if not verbose:
        !CUDA_VISIBLE_DEVICES=0 $sys.exec_prefix/bin/hd_glio_predict -t1 $t1 -t1c $t1c -t2 $t2 -flair $flair -o $output > /dev/null
    
    return output

...and run the already trained model on all our subjects:

In [ ]:
%%time
verbose=True

i=1
for subj in subject_ids:
    
    # Specify the file name for the output segmentation
    output_file = LOCAL_DATA/"TCGA"/subj/"seg"/f"{subj.split('/')[-1]}_seg.nii.gz"
    
    # Grab the file names for the four images (T1, T1c, T2, FLAIR)
    flair, t1, t1c, t2 = sorted(list((IMG_DATA/subj).glob("*.nii*")))
    
    # Run HDGLIO (note that it is possible to run HD-GLIO in batch mode, avoiding 
    # having to load the model every time. See the HD-GLIO docs for more)
    print(f"Processing subject #{i}/{len(subject_ids)}: {subj.split('/')[-1]}")
    
    run_hdglio(t1, t1c, t2, flair, output_file, verbose)
    
    print("-"*40)
    print(f"DONE. Output available at {output_file}")
    print()
    print("#"*40)
    print("#"*40)
    
    i+=1

## Inspect results

We've now produced segmentation masks for our tumor images. Here's a a small widget to plot the resulting segmentation masks and the corresponding T2 images.

In [ ]:
from ipywidgets import ToggleButtons, fixed

In [ ]:
classes_dict = {
    'Normal': 0.,
    'Contrast-enhancing': 1.,
    'Non-enhancing': 2.,
}

channel_dict = {
    'Flair': 0.,
    'T1': 1.,
    'T1c': 2.,
    'T2': 3
}

In [ ]:
# Plotting function
def plot_segmentation(subj_id, channel, seg_class, slice_nb):
    
    flair, t1, t1c, t2 = [nib.load(i) for i in sorted(list((IMG_DATA/subj_id).glob("*.nii*")))]
    mri_channels = {'Flair': flair, 'T1': t1, 'T1c': t1c, 'T2': t2}
    
    mask = nib.load(list((IMG_DATA/subj_id/"seg").glob("*.nii*"))[0])
    
    nb_slices = mri_channels[channel].shape[-1]
    
    img_data = np.flip(np.rot90(mri_channels[channel].get_fdata()[:,:,slice_nb]))
    mask_data = np.flip(np.rot90(mask.get_fdata()[:,:,slice_nb]))
    
    print(f"Label: {seg_class}")
    img_label = classes_dict[seg_class]
    mask = np.where(mask_data == img_label, 255, 0)
    
    f,axs = plt.subplots(1,3, figsize=(18,5))
    axs[0].imshow(img_data, cmap="gray")
    axs[0].axis('off')
    axs[1].imshow(mask, cmap='binary')
    axs[1].axis('off')
    axs[2].imshow(mask, cmap="Blues")
    axs[2].imshow(img_data, cmap="gray", alpha=0.7)
    axs[2].axis("off")

Set up the interactive elements of our widget:

In [ ]:
# Button
select_class = ToggleButtons(
    options=['Contrast-enhancing', 'Non-enhancing'],
    description='Select Label:',
    button_style='info', 
    
)
select_channel = ToggleButtons(
    options=list(channel_dict.keys()),
    description='Select Channel:',
    button_style='info', 
    
)

# Slice slider
select_layer = IntSlider(value=100, min=0, max=254, 
                         description='Slice', continuous_update=False)

# Subject selector
select_subj = Select(options=subject_ids, description="Subject")

In [ ]:
interactive(plot_segmentation, channel=select_channel, subj_id=select_subj,
            seg_class=select_class, slice_nb=select_layer)

# Radiomics: extracting tumor features

![](assets/radiomics.png)

Volume and location information for tumors is important, but there's a lot more information represented in the images. Information that could be useful for more precise diagnosis and prognosis. For example, the shape of the tumors, the tumors appearance in the different MRI contrast images, the heterogeneity in the various compartments of the tumors, and so on. As mentioned, such ideas leads us to **radiomics**.

There is a vast number of possible features to extract (at least several hundreds, depending on the number of imaging modalities), related to shape, intensity values and variation, and texture features from each of the MRI modalities. The robustness and relevance of the various features is somewhat unclear, but there's a lot of research into the clinical relevance of radiomics.

For demonstration purposes, we extract only the total volumes, a few shape features, and some texture information. More precisely, we extract:

1. The size of the enhancing and non-enhancing tumor
2. The maximum 3D diameter
3. Mean and variance of the intensities in the T1, T1c, T2, FLAIR in the enhancing and non-enhancing areas
4. The gray level nonuniformity in T1, T1c, T2, FLAIR.

### Data-driven science

> **Note:** It is interesting to contrast _data-driven_ and biologically and clinically inspired analyses. In principle, one can use and discover complicated features that are linked to important outcomes (e.g., survival) _driectly from the data_. Features that are potentially not understood from a biological point of view. This can, of course, also be quite dangerous, as one may uncover _spurious, meaningless patterns_ in the data one has collected. There will also be issues with _explainability_ if one drops the link to biological explanations. 

### RANO criteria for Glioblastoma Multiforme

<img src="assets/rano.png">

## Extracting features using PyRadiomics

To extract radiomic features, we use an established, open-source library developed by the imaging community, PyRadiomics. The library aims to be the reference standard for radiomic analysis, easing both analyses and enabling greater reproducibility. PyRadiomics is available on GitHub: https://github.com/AIM-Harvard/pyradiomics.

<img src="https://www.radiomics.io/img/pyrad1.jpg">

<img src="assets/pyradiomics.png">

We specify the features we'd like to extract in a file (YAML format):

In [ ]:
#%cat pyradiomics_settings.yaml

In [ ]:
from radiomics import featureextractor

In [ ]:
def get_radiomics(images, mask, params='pyradiomics_settings.yaml'):
    results = {}
    
    subject_id = images[0].stem.split("_")[0]
    results['SUBJECT ID'] = subject_id
    
    # Some features are the same across all individual MRI image sequences 
    # We can extract them from any sequence

    extractor = featureextractor.RadiomicsFeatureExtractor()
    extractor.loadParams(params)
    
    
    result_label1 = extractor.execute(str(images[0]), mask, label=1)
    result_label2 = extractor.execute(str(images[0]), mask, label=2)
        
    results[f'Volume_label1'] = result_label1['original_shape_MeshVolume'].round(2)
    results[f'Volume_label2'] = result_label2['original_shape_MeshVolume'].round(2)
        
    results[f'Max3DDiameter_label1'] = result_label1['original_shape_Maximum3DDiameter'].round(2)
    results[f'Max3DDiameter_label2'] = result_label2['original_shape_Maximum3DDiameter'].round(2)
    
    
    
    # Other features we extract from each individual type of MRI image

    for img in images:
        img_type = img.stem.split("_")[-1].split(".")[0]
        result_label1 = extractor.execute(str(img), mask, label=1)
        result_label2 = extractor.execute(str(img), mask, label=2)
        
        
        # Mean intensity
        
        results[f'{img_type}_MeanIntensity_label1'] = result_label1['original_firstorder_Mean'].round(2)
        results[f'{img_type}_MeanIntensity_label2'] = result_label2['original_firstorder_Mean'].round(2)
        
        # Intensity variance
        
        results[f'{img_type}_IntensityVariance_label1'] = result_label1['original_firstorder_Variance'].round(2)
        results[f'{img_type}_IntensityVariance_label2'] = result_label2['original_firstorder_Variance'].round(2)
        
        # Gray level nonuniformity
        
        results[f'{img_type}_GrayLevelNonUniformity_label1'] = result_label1['original_gldm_GrayLevelNonUniformity'].round(2)
        results[f'{img_type}_GrayLevelNonUniformity_label2'] = result_label2['original_gldm_GrayLevelNonUniformity'].round(2)       
        
        
    results = pd.DataFrame.from_dict(results, orient='index').T
    
    return results

## Run extraction on all subjects

In [ ]:
results = []
i=1
for subject_id in subject_ids:
    print(f"#{i}: Computing radiomic features for subject {subject_id}")
    images = sorted(list((LOCAL_DATA/'TCGA'/subject_id).glob('*.nii.gz')))
    mask = str(list((LOCAL_DATA/'TCGA'/subject_id/'seg').glob('*.nii.gz'))[0])
    res = get_radiomics(images, mask)
    results.append(res)
    i+=1

We construct a data frame that stores the resulting radiomic features:

In [ ]:
radiomic_df = pd.concat(results)

In [ ]:
radiomic_df

In [ ]:
radiomic_df['tumor_volume'] = radiomic_df['Volume_label1'] + radiomic_df['Volume_label2']

## Construct our final data set

Now we can append these features to the other information we have about each subject in the data set: 

In [ ]:
sample_df.head()

In [ ]:
df = pd.merge(sample_df, radiomic_df, on="SUBJECT ID")

We're left with quite an interesting data set:

In [ ]:
df

In [ ]:
df.to_csv(LOCAL_DATA/'tcga_radiomics_sample.csv', index=None)

In [ ]:
df = pd.read_csv(LOCAL_DATA/'tcga_radiomics_sample.csv')

# What's next?

It would be natural to investigate whether the radiomic features together with the other information we have about the subjects can provide relevant clinical information. In other words, to what extent the various features are associated to clinical outcomes (e.g., survival), either individually or together. This can be done using e.g., plots, basic statistics, statistical modelling, or machine learning.

Here's a plot of the length of survival for our subjects:

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(12,6))
ax = sns.scatterplot(x='SUBJECT ID', y='Survival(months)', hue='SUBJECT ID',
                     data=df.sort_values(by='Survival(months)'), s=60)
plt.xticks(rotation=45)
plt.show()

Here's the total tumor volume versus length of survival:

In [ ]:
plt.figure(figsize=(12,6))
sns.scatterplot(x='tumor_volume', y='Survival(months)', hue='SUBJECT ID',
                     data=df.sort_values(by='Survival(months)'), s=60)
plt.xticks(rotation=45)
plt.show()

We can bin the survival data into two: <= 5 months, > 5 months, and then study how this relates to the various radiomics features:

In [ ]:
df['short_survival'] = (df['Survival(months)'] <= 5).astype(str)

In [ ]:
df

Here's a plot investigating the relation between the IDH status, survival times and volume of the enhancing-non-enhancing tumor regions:

In [ ]:
plt.figure(figsize=(12,6))
sns.stripplot(data=df, x="short_survival", y="Volume_label1", hue='IDH-status',palette="vlag", s=10)
plt.show()

# Integrated diagnostics

Note that multiple sources of information beyond MRI could be valuable when assessing a glioblastoma case. A system tasked with extracting relevant, actionable information should therefore have access to more than the MRI images. 

This is a general principle in medicine: important information about a patient, disease, or condition is represented in a vast set of heterogeneous data. This leads to the need for **integrated diagnostics**. 

<img width=40% src="assets/fusion.png">

<img src="assets/integrated_diagnostics.png">

# What's next in the workshop?

Our sample data set is very small so we cannot say much at this point, but if you rerun this experiment using all the data available in TCGA-GBM you will have a data set that is quite large, comparably speaking. All the data is available through the TCGA database (after a simple access application process). You're therefore very close to being able to do some exciting research into brain tumors! 

## Image segmentation: the holy grail of medical image analysis

As you observed, a crucial component in the above story was the need for accurate image segmentation techniques. This gave us our _regions-of-interest_, without which we would've been unable to perform our analysis. This is a general principle in medical image analysis, and image segmentation is often called **the holy grail of medical image analysis**. 

We saw that **deep learning** gave us an approach to image segmentation. That is one reason–among many–we should take a closer look at deep learning. We turn to this next. 

# Your turn!

If you're interested in expanding the above radiomics analysis, I'm happy to discuss some possible starting points. One would f.ex. expand the data set and use a different segmentation model. And then conduct a careful statistical study of the various possible radiomic features (and also think about their their relevance).

Here are some pointers:

## Data
<a href="https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=24282668"><img src="assets/tcia-segm.png"></a>
<br><br>
<a href="https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=1966258"><img src="assets/tcga-gbm.png"></a>


## Model

<a href="https://github.com/rixez/brats21_kaist_mri_lab"><img src="assets/nnunet-seg-model.png"></a>

## Deployment

<a href="https://github.com/mmiv-center/Research-Information-System/tree/master/components/Workflow-Image-AI"><img src="assets/deploy.png"></a>